## Extract all lookup data

In [2]:
import { csvToJson } from './helpers/csvHelpers.ts'
import { CsvFields } from './helpers/csvTypes.ts'

const pathToBirthCsv = './sourceData/Birth_Register.csv'
const pathToDeathCsv = './sourceData/Death_Register.csv'
const pathToMarriageCsv = './sourceData/Marriage_Register.csv'
const pathToAdoptionCsv = './sourceData/Adoption_Register.csv'
const pathToDeedpollCsv = './sourceData/Deedpoll.csv'

const csvData: CsvFields = {
  birth: await csvToJson(pathToBirthCsv),
  death: await csvToJson(pathToDeathCsv),
  marriage: await csvToJson(pathToMarriageCsv),
  adoption: await csvToJson(pathToAdoptionCsv),
  deedpoll: await csvToJson(pathToDeedpollCsv),
}

const extractLookupFields = (
  register: Register,
  fields: RegisterField | RegisterField[],
  outputPath: string,
) => {

  const fieldsArray = Array.isArray(fields) ? fields : [fields]
  const values = register
    .flatMap((record) => fieldsArray.map((field) => record[field]))
    .filter((x) => x)
  const uniqueValues = Array.from(new Set(values)).sort()
  Deno.writeFileSync(
    outputPath,
    new TextEncoder().encode(JSON.stringify(uniqueValues, null, 2)),
  )
}




### Seeded Data


In [2]:
const seededLocations = await csvToJson(
  '../../ckopencrvs/src/data-seeding/locations/source/locations.csv',
)

Deno.writeFileSync(
  './lookupMappings/locationsSeeded.json',
  new TextEncoder().encode(JSON.stringify(seededLocations, null, 2)),
)

const facilities = await csvToJson(
  '../../ckopencrvs/src/data-seeding/locations/source/health-facilities.csv',
)
const facs = facilities.map((facility: any) => ({
  name: facility.name,
  id: facility.id,
}))

Deno.writeFileSync(
  './lookupMappings/facilities.json',
  new TextEncoder().encode(JSON.stringify(facs, null, 2)),
)

### Birth

In [3]:
import { locationsMap } from './lookupMappings/locations.ts'

// AI did this
const mappedUniqueBirthAddresses = locationsMap.map((location) => ({
  name: location.name,
  map: location.map,
  facilityCode: null,
  country: null,
  intlTown: null,
}))




In [4]:
extractLookupFields(
  csvData.birth,
  [
    'MOTHERS_ADDRESS',
    'MOTHERS_BIRTHPLACE',
    'INFORMANTS_ADDRESS',
    'CHILDS_BIRTHPLACE',
    'FATHERS_BIRTHPLACE',
    'FATHERS_ADDRESS',
  ] as RegisterField[],
  './lookupMappings/birth/addressesRaw.json',
)

extractLookupFields(
  csvData.birth,
  'INFORMANTS_RELATIONSHIP' as RegisterField,
  './lookupMappings/birth/informantTypesRaw.json',
)

extractLookupFields(
  csvData.birth,
  ['FATHERS_NATIONALITY', 'MOTHERS_NATIONALITY'] as RegisterField[],
  './lookupMappings/birth/nationalitiesRaw.json',
)

extractLookupFields(
  csvData.birth,
  ['FATHERS_RACE', 'MOTHERS_RACE'] as RegisterField[],
  './lookupMappings/birth/racesRaw.json',
)

extractLookupFields(
  csvData.birth,
  'CHILDS_TWIN' as RegisterField,
  './lookupMappings/birth/twinsRaw.json',
)

extractLookupFields(
  csvData.birth,
  'REGISTRAR' as RegisterField,
  './lookupMappings/birth/registrarsRaw.json',
)


### Deedpoll


In [5]:
extractLookupFields(
  csvData.deedpoll,
  ['ISLAND', 'WITNESS_ADDRESS'] as RegisterField[],
  './lookupMappings/deedpoll/addressesRaw.json',
)


### Death

In [ ]:
import { Register, RegisterField } from './helpers/csvTypes.ts'

extractLookupFields(
  csvData.death,
  [
    'WHERE_DIED',
    'USUAL_RESIDENCE',
    'WHERE_BURIED',
    'WHERE_BORN',
  ] as RegisterField[],
  './lookupMappings/death/addressesRaw.json',
)

extractLookupFields(
  csvData.death,
  'REGISTRAR' as RegisterField,
  './lookupMappings/death/registrarsRaw.json',
)

extractLookupFields(
  csvData.death,
  'LENGTH_IN_CIS' as RegisterField,
  './lookupMappings/death/lengthInCisRaw.json',
)

extractLookupFields(
  csvData.death,
  'INFORMANT_DESCRIPTION' as RegisterField,
  './lookupMappings/death/informantDescriptionRaw.json',
)



### Adoption


In [7]:
extractLookupFields(
  csvData.adoption,
  [
    'CHILDS_BIRTHPLACE',
    'FATHERS_ADDRESS',
    'MOTHERS_BIRTHPLACE',
    'MOTHERS_ADDRESS',
  ] as RegisterField[],
  './lookupMappings/adoption/addressesRaw.json',
)

extractLookupFields(
  csvData.adoption,
  'CHILDS_TWIN' as RegisterField,
  './lookupMappings/adoption/childsTwinRaw.json',
)

extractLookupFields(
  csvData.adoption,
  ['FATHERS_RACE', 'MOTHERS_RACE'] as RegisterField[],
  './lookupMappings/adoption/racesRaw.json',
)

extractLookupFields(
  csvData.adoption,
  ['FATHERS_NATIONALITY', 'MOTHERS_NATIONALITY'] as RegisterField[],
  './lookupMappings/adoption/nationalitiesRaw.json',
)

extractLookupFields(
  csvData.adoption,
  'REGISTRAR' as RegisterField,
  './lookupMappings/adoption/registrarsRaw.json',
)


### Marriage

In [8]:
extractLookupFields(
  csvData.marriage,
  [
    'MARRIAGE_PLACE',
    'GROOM_BIRTHPLACE',
    'BRIDE_BIRTHPLACE',
    'GROOM_RESIDENCE',
    'BRIDE_RESIDENCE',
    'Witness1_Address',
    'Witness2_Address',
    'ISLAND',
  ] as RegisterField[],
  './lookupMappings/marriage/addressesRaw.json',
)

extractLookupFields(
  csvData.marriage,
  'REGISTRAR' as RegisterField,
  './lookupMappings/marriage/registrarsRaw.json',
)


### Prep all similar types

In [ ]:
// Addresses

import { readJsonFile } from "./helpers/csvHelpers.ts";

const allAddresses = readJsonFile('./lookupMappings/birth/addressesRaw.json')
  .concat(readJsonFile('./lookupMappings/deedpoll/addressesRaw.json'))
  .concat(readJsonFile('./lookupMappings/death/addressesRaw.json'))
  .concat(readJsonFile('./lookupMappings/adoption/addressesRaw.json'))
  .concat(readJsonFile('./lookupMappings/marriage/addressesRaw.json'))

const uniqueAddresses = Array.from(new Set(allAddresses)).sort()

Deno.writeFileSync(
  './lookupMappings/allUniqueAddresses.json',
  new TextEncoder().encode(JSON.stringify(uniqueAddresses, null, 2)),
)


In [10]:
// Registrars

const allRegistrars = readJsonFile('./lookupMappings/birth/registrarsRaw.json')
  .concat(readJsonFile('./lookupMappings/death/registrarsRaw.json'))
  .concat(readJsonFile('./lookupMappings/adoption/registrarsRaw.json'))
  .concat(readJsonFile('./lookupMappings/marriage/registrarsRaw.json'))

const uniqueRegistrars = Array.from(new Set(allRegistrars)).sort()

Deno.writeFileSync(
  './lookupMappings/allUniqueRegistrars.json',
  new TextEncoder().encode(JSON.stringify(uniqueRegistrars, null, 2)),
)


In [11]:
// Nationalities

const allNationalities = readJsonFile('./lookupMappings/birth/nationalitiesRaw.json')
  .concat(readJsonFile('./lookupMappings/birth/racesRaw.json'))
  .concat(readJsonFile('./lookupMappings/adoption/nationalitiesRaw.json'))
  .concat(readJsonFile('./lookupMappings/adoption/racesRaw.json'))

const uniqueNationalities = Array.from(new Set(allNationalities)).sort()

Deno.writeFileSync(
  './lookupMappings/allUniqueNationalitiesAndRaces.json',
  new TextEncoder().encode(JSON.stringify(uniqueNationalities, null, 2)),
)